In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn import metrics
import pandas_datareader.data as web 
from time import time

#Read stock data use pandas_datareader.data from web
#Get Stock Data

In [24]:
stk_tickers = ['MSFT', 'IBM', 'GOOGL']
ccy_tickers = ['DEXJPUS', 'DEXUSUK']
idx_tickers = ['SP500', 'DJIA', 'VIXCLS']
stk_data = web.DataReader(stk_tickers, 'yahoo')
ccy_data = web.DataReader(ccy_tickers, 'fred')
idx_data = web.DataReader(idx_tickers, 'fred')

In [25]:
#Select columns
base = stk_data.loc[:, ('Adj Close', 'MSFT')]
X1 = stk_data.loc[:, ('Adj Close', ('GOOGL', 'IBM'))]
X2 = ccy_data
X3 = idx_data
print(X1)
print(X2)
print(X2)

Attributes   Adj Close            
Symbols          GOOGL         IBM
Date                              
2017-09-21   47.377499  109.482422
2017-09-22   47.162998  109.384438
2017-09-25   46.714001  109.942169
2017-09-26   46.871498  110.462234
2017-09-27   47.994999  109.783905
...                ...         ...
2022-09-14  105.000000  127.690002
2022-09-15  102.910004  125.489998
2022-09-16  102.800003  127.269997
2022-09-19  103.070000  127.730003
2022-09-20  101.139999  126.300003

[1258 rows x 2 columns]
            DEXJPUS  DEXUSUK
DATE                        
2017-09-22   112.01   1.3531
2017-09-25   111.64   1.3457
2017-09-26   112.16   1.3422
2017-09-27   112.76   1.3401
2017-09-28   112.66   1.3435
...             ...      ...
2022-09-12   142.41   1.1701
2022-09-13   144.30   1.1526
2022-09-14   142.93   1.1564
2022-09-15   143.57   1.1472
2022-09-16   143.04   1.1419

[1301 rows x 2 columns]
            DEXJPUS  DEXUSUK
DATE                        
2017-09-22   112.01   1.3

In [26]:
#Standardized data (X1, X2, X3) with kept index (date)
standard_scaler = preprocessing.StandardScaler()
x1 = standard_scaler.fit_transform(X1.values)
X1 = pd.DataFrame(data=x1,index=X1.index,columns=X1.columns)
X1

Attributes Adj Close          
Symbols        GOOGL       IBM
Date                          
2017-09-21 -1.135074 -0.550679
2017-09-22 -1.141863 -0.559724
2017-09-25 -1.156072 -0.508238
2017-09-26 -1.151088 -0.460230
2017-09-27 -1.115532 -0.522848
...              ...       ...
2022-09-14  0.688514  1.130117
2022-09-15  0.622372  0.927028
2022-09-16  0.618890  1.091345
2022-09-19  0.627435  1.133810
2022-09-20  0.566356  1.001802

[1258 rows x 2 columns]

#Calculate ความแตกต่างของค่า ราคา 'Adj Close', 'MSFT’)ย้อนหลัง backHisotry วัน

In [27]:

backHistory = [30, 45, 60, 90, 180, 240] #-> ทดลองหยิบ 3 ค่า 3 รูปแบบ เพื่อดูระยะเวลาการดูค่าข้อมูลย้อนหลงัหลายๆแบบและเปรียบเทียบ MSE
BH1, BH2, BH3 = backHistory[1], backHistory[3], backHistory[4]
return_period = 1
Y = base.shift(-return_period)
X4_BH1 = base.diff( BH1).shift( - BH1)
X4_BH2 = base.diff( BH2).shift( - BH2)
X4_BH3 = base.diff( BH3).shift( - BH3)

X4 = pd.concat([X4_BH1, X4_BH2, X4_BH3], axis=1)
X4.columns = ['MSFT_3DT', 'MSFT_6DT', 'MSFT_12DT']
X4 = pd.DataFrame(standard_scaler.fit_transform(X4.values), index = X4.index,columns=X4.columns)

In [32]:
# Forming Dataset
X = pd.concat([X1, X2, X3, X4], axis=1)
dataset = pd.concat([Y, X], axis=1)
dataset.dropna(inplace=True)
dataset.describe()

,"(Adj Close, MSFT)","(Adj Close, GOOGL)","(Adj Close, IBM)",DEXJPUS,DEXUSUK,SP500,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
count,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000
mean,165.351737,-0.207698,-0.224548,109.247613,1.318670,3242.391849,27799.497453,19.617887,0.130186,0.156007,0.001622
std,73.033812,0.916802,0.871623,2.733256,0.053741,638.350141,3905.028923,8.985684,0.881999,0.862664,0.994174
min,68.838753,-1.156072,-3.286414,102.520000,1.149200,2237.400000,18591.930000,9.140000,-3.843044,-3.688342,-3.727184
25%,102.613165,-0.860310,-0.697378,107.300000,1.283825,2760.082500,25015.377500,13.740000,-0.269621,-0.234484,-0.420715
50%,140.262482,-0.645754,-0.249923,109.345000,1.314050,2974.480000,26467.715000,17.380000,0.089257,0.086493,0.048350
75%,214.331333,0.138415,0.260329,111.222500,1.363175,3667.065000,29986.780000,22.525000,0.531038,0.589458,0.627076
max,340.882812,2.107525,1.900371,115.340000,1.433200,4793.060000,36488.630000,82.690000,2.580097,2.211056,2.443553
